In [247]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.preprocessing import LabelEncoder, StandardScaler
import json
import urllib
import requests
import datetime

In [248]:
#Get Path
os.getcwd()
basepath = os.getcwd()
print(basepath)
lis_dir = os.listdir()
print(lis_dir)


C:\Users\Tiki\Documents\Sprinboard2\Capstone_cars
['.ipynb_checkpoints', '575_1100_compressed_data.csv', '575_1100_compressed_data.csv.zip', '62920_1026452_bundle_archive.zip', 'auto1.ipynb', 'auto1_datawrangling.ipynb', 'auto2.ipynb', 'auto3.ipynb', 'Combining Data.ipynb', 'correlation.png', 'data']


In [249]:
# Create the list of file names: filenames
file ='auto_data.csv'
sub_file = 'data'
print('p')
file_name_path = os.path.join(basepath, sub_file, file)
print('y')
auto_data= pd.read_csv(file_name_path)
#Open data
auto_data.head(5)

p
y


,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,age
0,7088744126,https://greensboro.craigslist.org/cto/d/greens...,greensboro,https://greensboro.craigslist.org,9500,2011.0,jaguar,xf,excellent,NaN,...,NaN,NaN,blue,https://images.craigslist.org/00505_f22HGItCRp...,2011 jaguar XF premium - estate sale. Retired ...,NaN,nc,36.1032,-79.8794,9.0
1,7093461424,https://denver.craigslist.org/ctd/d/westminste...,denver,https://denver.craigslist.org,7995,2010.0,chevrolet,silverado 1500 4wd,excellent,8.0,...,full-size,truck,white,https://images.craigslist.org/00u0u_8Z0ctz08dr...,This Silverado work truck must've been used by...,NaN,co,39.8302,-105.0370,10.0
2,7088708344,https://greensboro.craigslist.org/ctd/d/greens...,greensboro,https://greensboro.craigslist.org,15300,2015.0,honda,accord lx-s coupe 2d,good,NaN,...,NaN,coupe,black,https://images.craigslist.org/00g0g_3FpqFaExYI...,Welcome to Carvana—the new way to buy a car (e...,NaN,nc,36.0700,-79.8200,5.0
3,7076226877,https://syracuse.craigslist.org/cto/d/syracuse...,syracuse,https://syracuse.craigslist.org,10950,2011.0,buick,lucerne cxl v6,excellent,6.0,...,full-size,sedan,red,https://images.craigslist.org/00r0r_dBDjHZSiKK...,"Grandma's 2011 Buick Lucerne. 43,418 babied m...",NaN,ny,43.1226,-76.1284,9.0
4,7076224988,https://syracuse.craigslist.org/cto/d/cicero-2...,syracuse,https://syracuse.craigslist.org,9400,2011.0,bmw,x5,good,6.0,...,NaN,SUV,blue,https://images.craigslist.org/00p0p_hdJbqFNwf3...,Selling a 2011 BMW X5 in mint condition with f...,NaN,ny,43.1707,-76.0962,9.0


In [250]:
auto_data= auto_data.drop(columns=['url', 'region_url', 'image_url', 'county'], axis=1)


In [251]:
print(f"Maximum mileage: {auto_data.odometer.max()} \nMinimum mileage: {auto_data.odometer.min()} ")
# If we are dealing with a vehicle with over a half milion miles we have more problem

auto_data = auto_data[(auto_data.odometer< 500000)] 

print(f"Maximum price: $ {auto_data.price.max()} \nMinimum price: $ {auto_data.price.min()} ")

#None of the vehicles we have even brand new have a value of half a milion
#We will also delete the cars with a price less than $100, because it might be a mistake if the price is so low.
auto_data = auto_data[(auto_data.price >= 100) & (auto_data.price <= 300000)]


print(f"Maximum age:  {auto_data.age.max()} years \nMinimum age: {auto_data.age.min()} years ")

Maximum mileage: 499103.0 
Minimum mileage: 0.0 
Maximum price: $ 450000 
Minimum price: $ 100 
Maximum age:  10.0 years 
Minimum age: 0.0 years 


In [252]:
#Create mmy_a # column
auto_data.columns

#Fix manufacturer
auto_data['manufacturer']= auto_data['manufacturer'].str.strip() 
auto_data['manufacturer']= auto_data['manufacturer'].str.replace('land rover' , 'land-rover') 
auto_data['manufacturer']= auto_data['manufacturer'].str.replace('rover' , 'land-rover') 
auto_data['manufacturer']= auto_data['manufacturer'].str.replace('land-land' , 'land')
auto_data['manufacturer']= auto_data['manufacturer'].str.replace('rover' , 'land-rover') 
auto_data['manufacturer']= auto_data['manufacturer'].str.replace('porche', 'porsche')
auto_data['manufacturer']= auto_data['manufacturer'].str.replace(' ', '-')

#fix Model
auto_data['model']= auto_data['model'].str.strip() 
auto_data['modelx']= auto_data['model'].str.replace(' ', '-')
auto_data['modelx']= auto_data['model'].str.replace('---', '-')
auto_data['modelx']= auto_data['model'].str.replace('--', '-')





In [253]:
#crreate key
#auto_data['mmy_a']=auto_data['manufacturer'].str.strip()+' '+auto_data['modelx'].str.strip()+ ' '+ auto_data['year'].astype(int).astype(str)
auto_data['mmy_a']=auto_data['manufacturer'].str.strip()+' '+auto_data['model'].str.strip()+ ' '+ auto_data['year'].astype(int).astype(str)
auto_data['mmy_a']=auto_data['mmy_a'].str.lower()

#auto_data['model2']= auto_data['model'].str.strip().str.split(' ').str[0]+'-'+auto_data['model'].str.strip().str.split(' ').str[1]
auto_data['model2']= auto_data['model'].str.strip().str.split(' ').str[0]+' '+auto_data['model'].str.strip().str.split(' ').str[1]

auto_data['model2']= auto_data['model2'].str.strip()
auto_data['mmy_a2']=auto_data['manufacturer']+' '+auto_data['model2']+ ' '+ auto_data['year'].astype(int).astype(str)
auto_data['mmy_a2']= auto_data['mmy_a2'].str.lower()
auto_data.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,...,paint_color,description,state,lat,long,age,modelx,mmy_a,model2,mmy_a2
0,7088744126,greensboro,9500,2011.0,jaguar,xf,excellent,NaN,gas,85000.0,...,blue,2011 jaguar XF premium - estate sale. Retired ...,nc,36.1032,-79.8794,9.0,xf,jaguar xf 2011,NaN,NaN
1,7093461424,denver,7995,2010.0,chevrolet,silverado 1500 4wd,excellent,8.0,gas,194050.0,...,white,This Silverado work truck must've been used by...,co,39.8302,-105.0370,10.0,silverado 1500 4wd,chevrolet silverado 1500 4wd 2010,silverado 1500,chevrolet silverado 1500 2010
2,7088708344,greensboro,15300,2015.0,honda,accord lx-s coupe 2d,good,NaN,gas,54296.0,...,black,Welcome to Carvana—the new way to buy a car (e...,nc,36.0700,-79.8200,5.0,accord lx-s coupe 2d,honda accord lx-s coupe 2d 2015,accord lx-s,honda accord lx-s 2015
3,7076226877,syracuse,10950,2011.0,buick,lucerne cxl v6,excellent,6.0,gas,43418.0,...,red,"Grandma's 2011 Buick Lucerne. 43,418 babied m...",ny,43.1226,-76.1284,9.0,lucerne cxl v6,buick lucerne cxl v6 2011,lucerne cxl,buick lucerne cxl 2011
4,7076224988,syracuse,9400,2011.0,bmw,x5,good,6.0,gas,145000.0,...,blue,Selling a 2011 BMW X5 in mint condition with f...,ny,43.1707,-76.0962,9.0,x5,bmw x5 2011,NaN,NaN


In [254]:
#find summary statistics
auto_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142138 entries, 0 to 142140
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            142138 non-null  int64  
 1   region        142138 non-null  object 
 2   price         142138 non-null  int64  
 3   year          142138 non-null  float64
 4   manufacturer  139154 non-null  object 
 5   model         142138 non-null  object 
 6   condition     142138 non-null  object 
 7   cylinders     103418 non-null  float64
 8   fuel          142128 non-null  object 
 9   odometer      142138 non-null  float64
 10  title_status  142132 non-null  object 
 11  transmission  142006 non-null  object 
 12  vin           103835 non-null  object 
 13  drive         118448 non-null  object 
 14  size          56919 non-null   object 
 15  type          127086 non-null  object 
 16  paint_color   118150 non-null  object 
 17  description   142136 non-null  object 
 18  stat

## Load MSRP Data

In [255]:
# Create the list of file names: filenames
file_m ='msrp.csv'
sub_file = 'data'
file_name_path_m = os.path.join(basepath, sub_file, file_m)
msrp_data= pd.read_csv(file_name_path_m)
#Open data
msrp_data.head(5)


,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [256]:
print(f"Maximum MSRP: {msrp_data.MSRP.max()} \nMinimum MSRP: {msrp_data.MSRP.min()} ")
# If we are dealing with a vehicle with an MSRP over half milion miles we have more problem
msrp_data = msrp_data[(msrp_data.MSRP< 500000 )] 
print(f"Maximum Year: {msrp_data.Year.max()} \nMinimum Year: {msrp_data.Year.min()} ")
# We only need vehicles with over a 2009 MY
msrp_data = msrp_data[(msrp_data.Year> 2009 )] 

Maximum MSRP: 497650 
Minimum MSRP: 11965 
Maximum Year: 2017 
Minimum Year: 2010 


In [257]:
msrp_data.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

In [258]:
msrp_data.columns
msrp_data['Make']= msrp_data['Make'].str.strip() 
msrp_data['Make']= msrp_data['Make'].str.replace(' ', '-')
#msrp_data['Modelx']= msrp_data['Model'].str.strip() 
#msrp_data['Modelx']= msrp_data['Modelx'].str.replace(' ', '-')
#msrp_data['mmy_m']=msrp_data['Make'].str.strip()+' '+msrp_data['Modelx'].str.strip()+ ' '+ msrp_data['Year'].astype(int).astype(str)
msrp_data['mmy_m']=msrp_data['Make'].str.strip()+' '+msrp_data['Model'].str.strip()+ ' '+ msrp_data['Year'].astype(int).astype(str)
msrp_data['mmy_m']= msrp_data['mmy_m'].str.lower()
msrp_data['mmy_m'].head()

0    bmw 1 series m 2011
1      bmw 1 series 2011
2      bmw 1 series 2011
3      bmw 1 series 2011
4      bmw 1 series 2011
Name: mmy_m, dtype: object

In [259]:
MSRP_dict = msrp_data.groupby(['Make', 'Model', 'Year', 'mmy_m'])['MSRP'].mean().reset_index()


## Combine Auto key and MSRP key

In [260]:
df1= auto_data[['manufacturer', 'model', 'model2', 'year', 'mmy_a2', 'mmy_a']]
df1 = df1[df1.manufacturer != 'harley-davidson']
df1 = df1[df1.manufacturer != 'hennessey']
df1= df1.drop_duplicates(keep='last')
df2= MSRP_dict

In [261]:
a_m_dict= pd.merge(df1, df2, left_on='mmy_a', right_on= 'mmy_m', how='left' )
a_m_dict.info()
a_m_dict =a_m_dict.dropna()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23464 entries, 0 to 23463
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  21736 non-null  object 
 1   model         23464 non-null  object 
 2   model2        20176 non-null  object 
 3   year          23464 non-null  float64
 4   mmy_a2        18638 non-null  object 
 5   mmy_a         21736 non-null  object 
 6   Make          855 non-null    object 
 7   Model         855 non-null    object 
 8   Year          855 non-null    float64
 9   mmy_m         855 non-null    object 
 10  MSRP          855 non-null    float64
dtypes: float64(3), object(8)
memory usage: 2.1+ MB


In [262]:
a_m_dict_append= pd.merge(df1, df2, left_on='mmy_a2', right_on= 'mmy_m', how='left' )
a_m_dict_append.info()
a_m_dict_append =a_m_dict_append.dropna()
a_m_dict= a_m_dict.append(a_m_dict_append)
a_m_dict = a_m_dict.drop_duplicates(subset=['mmy_a'], keep='first')



<class 'pandas.core.frame.DataFrame'>
Int64Index: 23464 entries, 0 to 23463
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  21736 non-null  object 
 1   model         23464 non-null  object 
 2   model2        20176 non-null  object 
 3   year          23464 non-null  float64
 4   mmy_a2        18638 non-null  object 
 5   mmy_a         21736 non-null  object 
 6   Make          820 non-null    object 
 7   Model         820 non-null    object 
 8   Year          820 non-null    float64
 9   mmy_m         820 non-null    object 
 10  MSRP          820 non-null    float64
dtypes: float64(3), object(8)
memory usage: 2.1+ MB


In [263]:
a_m_dict.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 828 entries, 53 to 23438
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  828 non-null    object 
 1   model         828 non-null    object 
 2   model2        828 non-null    object 
 3   year          828 non-null    float64
 4   mmy_a2        828 non-null    object 
 5   mmy_a         828 non-null    object 
 6   Make          828 non-null    object 
 7   Model         828 non-null    object 
 8   Year          828 non-null    float64
 9   mmy_m         828 non-null    object 
 10  MSRP          828 non-null    float64
dtypes: float64(3), object(8)
memory usage: 77.6+ KB


## Cat Data

In [264]:
sub_file = 'data'
new_file = 'cat_data.csv'
cat_file_name_path = os.path.join(basepath,sub_file, new_file)
cat_data= pd.read_csv(cat_file_name_path)
cat_data.head()


,Year,Make,Model,Category,updatedAt
0,2020,Audi,Q3,SUV,2020-01-27T20:44:17.665Z
1,2020,Cadillac,Escalade ESV,SUV,2020-01-27T20:44:17.665Z
2,2020,Chevrolet,Malibu,Sedan,2020-01-27T20:44:17.665Z
3,2020,Chevrolet,Corvette,"Coupe, Convertible",2020-01-27T20:44:17.665Z
4,2020,Acura,RLX,Sedan,2020-01-27T20:44:17.665Z


In [265]:
#Create Cat key
cat_data['make'] = cat_data['Make'].str.lower()
cat_data['make'] = cat_data['make'].str.strip()
cat_data['make']= cat_data['make'].str.replace(' ', '-') 
cat_data['mmy']=cat_data['make']+' '+cat_data['Model'].str.strip()+ ' '+ cat_data['Year'].astype(int).astype(str)
cat_data['mmy']= cat_data['mmy'].str.lower()


In [266]:
#merge MSRP cat
msrp_dict2 = MSRP_dict[['mmy_m', 'MSRP']]
cat_data= pd.merge(cat_data, msrp_dict2, left_on='mmy', right_on= 'mmy_m', how='left')

In [267]:
cat_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4082 entries, 0 to 4081
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Year       4082 non-null   int64  
 1   Make       4082 non-null   object 
 2   Model      4082 non-null   object 
 3   Category   4082 non-null   object 
 4   updatedAt  4082 non-null   object 
 5   make       4082 non-null   object 
 6   mmy        4082 non-null   object 
 7   mmy_m      959 non-null    object 
 8   MSRP       959 non-null    float64
dtypes: float64(1), int64(1), object(7)
memory usage: 318.9+ KB


In [268]:
## Merge Cat Data

In [269]:
#create mergeable sets
df1= auto_data[['manufacturer', 'model', 'year', 'mmy_a', 'mmy_a2']]
df1= df1.drop_duplicates(keep='last')
df1 = df1[df1.manufacturer != 'harley-davidson']
df1 = df1[df1.manufacturer != 'hennessey']
df2= cat_data[['Make',  'make', 'Model', 'Year', 'mmy', 'Category', 'MSRP']]
df2= df2.drop_duplicates(keep='last')

In [270]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23464 entries, 3 to 142140
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  21736 non-null  object 
 1   model         23464 non-null  object 
 2   year          23464 non-null  float64
 3   mmy_a         21736 non-null  object 
 4   mmy_a2        18638 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.1+ MB


In [271]:
#merge1
z= pd.merge(df1, df2, left_on='mmy_a', right_on='mmy', how='left')
z.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23464 entries, 0 to 23463
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  21736 non-null  object 
 1   model         23464 non-null  object 
 2   year          23464 non-null  float64
 3   mmy_a         21736 non-null  object 
 4   mmy_a2        18638 non-null  object 
 5   Make          2254 non-null   object 
 6   make          2254 non-null   object 
 7   Model         2254 non-null   object 
 8   Year          2254 non-null   float64
 9   mmy           2254 non-null   object 
 10  Category      2254 non-null   object 
 11  MSRP          685 non-null    float64
dtypes: float64(3), object(9)
memory usage: 2.3+ MB


In [272]:
dict_cat = z[['manufacturer', 'model', 'year', 'mmy_a' , 'Make', 'Model', 'Year', 'Category' ]].dropna()
dict_cat_m = z[['manufacturer', 'model', 'year', 'mmy_a' , 'Make', 'Model', 'Year', 'Category', 'MSRP' ]].dropna()
dict_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2254 entries, 5 to 23463
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  2254 non-null   object 
 1   model         2254 non-null   object 
 2   year          2254 non-null   float64
 3   mmy_a         2254 non-null   object 
 4   Make          2254 non-null   object 
 5   Model         2254 non-null   object 
 6   Year          2254 non-null   float64
 7   Category      2254 non-null   object 
dtypes: float64(2), object(6)
memory usage: 158.5+ KB


In [273]:
#merge2
z= pd.merge(df1, df2, left_on='mmy_a2', right_on= 'mmy', how='left' )
dict_cat_append= z[['manufacturer', 'model', 'year', 'mmy_a' , 'mmy', 'Make', 'Model', 'Year', 'Category' ]].dropna()
dict_cat_m_append= z[['manufacturer', 'model', 'year', 'mmy_a' ,  'mmy', 'Make', 'Model', 'Year', 'Category', 'MSRP' ]].dropna()



In [274]:
#append dictionary
dict_cat= dict_cat.append(dict_cat_append)
dict_cat_m= dict_cat_m.append(dict_cat_m_append)
dict_cat= dict_cat.drop_duplicates(keep='last')
dict_cat_m= dict_cat_m.drop_duplicates(keep='last')
dict_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3530 entries, 5 to 23449
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  3530 non-null   object 
 1   model         3530 non-null   object 
 2   year          3530 non-null   float64
 3   mmy_a         3530 non-null   object 
 4   Make          3530 non-null   object 
 5   Model         3530 non-null   object 
 6   Year          3530 non-null   float64
 7   Category      3530 non-null   object 
 8   mmy           1276 non-null   object 
dtypes: float64(2), object(7)
memory usage: 275.8+ KB


In [275]:
#combine dictionary
dict_cat_s= pd.merge(dict_cat, dict_cat_m, on=['manufacturer', 'model', 'year', 'mmy_a', 'Make', 'Model', 'Year',
       'Category', 'mmy'], how='left')
dict_cat_s.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3530 entries, 0 to 3529
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  3530 non-null   object 
 1   model         3530 non-null   object 
 2   year          3530 non-null   float64
 3   mmy_a         3530 non-null   object 
 4   Make          3530 non-null   object 
 5   Model         3530 non-null   object 
 6   Year          3530 non-null   float64
 7   Category      3530 non-null   object 
 8   mmy           1276 non-null   object 
 9   MSRP          1138 non-null   float64
dtypes: float64(3), object(7)
memory usage: 303.4+ KB


In [276]:
a_m_dict.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 828 entries, 53 to 23438
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  828 non-null    object 
 1   model         828 non-null    object 
 2   model2        828 non-null    object 
 3   year          828 non-null    float64
 4   mmy_a2        828 non-null    object 
 5   mmy_a         828 non-null    object 
 6   Make          828 non-null    object 
 7   Model         828 non-null    object 
 8   Year          828 non-null    float64
 9   mmy_m         828 non-null    object 
 10  MSRP          828 non-null    float64
dtypes: float64(3), object(8)
memory usage: 77.6+ KB


In [277]:
a_m_dict_s= a_m_dict[['mmy_a', 'MSRP']].drop_duplicates(keep='last')

In [278]:
#combine Cat dat and MSRP data again in dictionary
dict_cat_t= pd.merge(dict_cat_s, a_m_dict_s, on=['mmy_a'], how='left')
dict_cat_t.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3530 entries, 0 to 3529
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  3530 non-null   object 
 1   model         3530 non-null   object 
 2   year          3530 non-null   float64
 3   mmy_a         3530 non-null   object 
 4   Make          3530 non-null   object 
 5   Model         3530 non-null   object 
 6   Year          3530 non-null   float64
 7   Category      3530 non-null   object 
 8   mmy           1276 non-null   object 
 9   MSRP_x        1138 non-null   float64
 10  MSRP_y        581 non-null    float64
dtypes: float64(4), object(7)
memory usage: 330.9+ KB


In [279]:
dict_cat_h= dict_cat_t
dict_cat_h['MSRP'] = dict_cat_h['MSRP_x'].fillna(dict_cat_h['MSRP_y'])
dict_cat_h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3530 entries, 0 to 3529
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  3530 non-null   object 
 1   model         3530 non-null   object 
 2   year          3530 non-null   float64
 3   mmy_a         3530 non-null   object 
 4   Make          3530 non-null   object 
 5   Model         3530 non-null   object 
 6   Year          3530 non-null   float64
 7   Category      3530 non-null   object 
 8   mmy           1276 non-null   object 
 9   MSRP_x        1138 non-null   float64
 10  MSRP_y        581 non-null    float64
 11  MSRP          1153 non-null   float64
dtypes: float64(5), object(7)
memory usage: 358.5+ KB


In [280]:
pop = dict_cat_h.pop('MSRP_x')
pop = dict_cat_h.pop('MSRP_y')
dict_cat_h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3530 entries, 0 to 3529
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  3530 non-null   object 
 1   model         3530 non-null   object 
 2   year          3530 non-null   float64
 3   mmy_a         3530 non-null   object 
 4   Make          3530 non-null   object 
 5   Model         3530 non-null   object 
 6   Year          3530 non-null   float64
 7   Category      3530 non-null   object 
 8   mmy           1276 non-null   object 
 9   MSRP          1153 non-null   float64
dtypes: float64(3), object(7)
memory usage: 303.4+ KB


In [281]:
pop = dict_cat_h.pop('mmy')

In [282]:
#slim dictionary
#slim dictionary
dict_cat_h= dict_cat_h.drop_duplicates(keep='first')

dict_cat_h= dict_cat_h.drop_duplicates(keep='first')
dict_cat_h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3228 entries, 0 to 3528
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  3228 non-null   object 
 1   model         3228 non-null   object 
 2   year          3228 non-null   float64
 3   mmy_a         3228 non-null   object 
 4   Make          3228 non-null   object 
 5   Model         3228 non-null   object 
 6   Year          3228 non-null   float64
 7   Category      3228 non-null   object 
 8   MSRP          1044 non-null   float64
dtypes: float64(3), object(6)
memory usage: 252.2+ KB


In [283]:
dict_cat_h = dict_cat_h[['mmy_a', 'Category', 'MSRP']]
dict_cat_h = dict_cat_h.sort_values('MSRP',na_position='first')
dict_cat_h = dict_cat_h.drop_duplicates(subset=['mmy_a'], keep='last')
dict_cat_h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3221 entries, 0 to 616
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   mmy_a     3221 non-null   object 
 1   Category  3221 non-null   object 
 2   MSRP      1041 non-null   float64
dtypes: float64(1), object(2)
memory usage: 100.7+ KB


In [284]:
#pull dictionary to auto
auto_data= pd.merge(auto_data, dict_cat_h, on='mmy_a', how='left')
auto_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142138 entries, 0 to 142137
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            142138 non-null  int64  
 1   region        142138 non-null  object 
 2   price         142138 non-null  int64  
 3   year          142138 non-null  float64
 4   manufacturer  139154 non-null  object 
 5   model         142138 non-null  object 
 6   condition     142138 non-null  object 
 7   cylinders     103418 non-null  float64
 8   fuel          142128 non-null  object 
 9   odometer      142138 non-null  float64
 10  title_status  142132 non-null  object 
 11  transmission  142006 non-null  object 
 12  vin           103835 non-null  object 
 13  drive         118448 non-null  object 
 14  size          56919 non-null   object 
 15  type          127086 non-null  object 
 16  paint_color   118150 non-null  object 
 17  description   142136 non-null  object 
 18  stat

In [285]:
#Save data as new file
new_file = 'auto_data_complete.csv'
new_file_name_path = os.path.join(basepath,sub_file, new_file)
auto_data.to_csv(new_file_name_path, index=False)